# Задание

Рассмотрите по вариантам следующие задачи оптимизации:
* Log-optimal investment strategy without the constraint $x≥0$ ([ex. 4.60, p. 209](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf#page=223) and [10.14, p. 559](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf#page=573));
* Equality constrained analytic centering ([p. 548](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf#page=562));

$\to$ Equality constrained entropy maximization ([10.9, p. 558](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf#page=572));
* Minimizing a separable function subject to an equality constraint, $f_i(x_i) = x_i^4$, $i∈\{1, ..., n\}$ ([ex. 5.4, p. 248](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf#page=262));
* Optimal allocation with resource constraint, $f_i(x_i) = a_ie^{x_i} , a_i > 0$, $i ∈\{1, ..., n\}$ ([ex. 10.1, p. 523](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf#page=537)).

Дана следующая задача: $f(x) = \sum_{i=1}^{n}x_i log_2 x_i \to min$.

При ограничении: $Ax=b$.

Где $x ∈ R_{++}^2$, $A ∈ R^{m*n}$

1) Исследуйте задачу на выпуклость. Запишите необходимые условия минимума и двойственную задачу. 
2) Для каждого значения размерности $n ∈ \{10, 20, ..., 100\}$ сгенерируйте $N = 100$ тестовых примеров (необходимо проверять, чтобы целевая функция на допустимом множестве была ограни-
чена снизу). В каждом случае найдите глобальный минимум, $x^∗ ∈R^n$, с помощью CVX.
3) Для каждого значения $n ∈ \{10, 20, ..., 100\}$ и для каждого тестового примера сгенерируйте 100 начальных точек. Для заданной точности $ε = 0.01$ по значению функции решите задачу с помощью прямого и двойственного метода Ньютона (стандартный метод Ньютона для решения двойственной задачи). Приведите необходимые аналитические вычисления.
4) В качестве результата работы метода:
    * Для каждого метода и значений $n ∈\{10, 20, ..., 100\}$ среднее время работы метода и среднее число итераций (усреднение проводится по всем начальным точкам и по всем тестовым примерам). Сколько арифметических операций требуется для выполнения одной итерации метода?
    * Для одного тестового примера при $n = 10$ и нескольких различных начальных точек постройте зависимость точности по значению функции от числа итераций. Сравните полученные результаты для прямого и двойственного метода.

# Настройки/Импорты

Версии важных модулей:
* cvxpy==1.4.3
* numpy==1.23.0

In [2]:
import cvxpy as cp # солвер для задач
import numpy as np # для работы с массивами



In [21]:
n = np.arange(10, 101, 10) # возможные значения n (число переменных в задаче ~ размерность пространства) от 10 до 100 включительно
m = (n/2).astype(np.int32) # первая размерность матрицы A (должна быть меньше n, например, в два раза меньше n)
N = 100 # число тестовых примеров для каждого значения n
P = 100 # число начальных точек для каждого примера N
ε = 0.01 # необходимая точность

DIM = 10 # интересующая нас размерность пространства, на которой будут проходить тесты

# Вспомогательные функции

Целевая функция $f(x) = \sum_{i=1}^{n}x_i log_2 x_i \to min$, где $x ∈ R_{++}^2$, $A ∈ R^{m*n}$. <br>
Что аналогично матричному виду: $x^T log_2 x \to min$.

Её ограничение: $Ax=b$.

Производная: $∇f(x) = log_2 x + \frac{1}{ln2}$.

Гессиан: $∇^2f(x) = \frac{1}{xln2}$.

In [29]:
def func(x: np.array) -> np.float32:
    """
    Функция из задачи.\n
    Parameters:
        * x: текущие значения x (в виде столбца)\n
    Returns:
        * np.float32: значение функции в точке x
    """
    return x.T @ np.log2(x)

In [ ]:
def func_grad(x: np.array) -> np.array:
    """
    Производная функции из задачи.\n
    Parameters:
        * x: текущие значения x (в виде столбца)\n
    Returns:
        * np.array: вектор-столбец градиента функции в точке x
    """
    return np.log2(x) + 1/np.log(2)

In [13]:
def func_hessian(x: np.array) -> np.array:
    """
    Гессиан (матрица вторых производных) функции из задачи.\n
    Parameters:
        * x: текущие значения x (в виде столбца)\n
    Returns:
        * np.array: матрица вторых производных функции в точке x
    """
    return 1/(x*np.log(2))

In [ ]:
def constraints(x: np.array, A: np.array, b: np.array) -> bool:
    """
    Функция для проверка решения на допустимость.\n
    Parameters:
        * x: текущие значения x (в виде вектора-столбца)
        * A: матрица A
        * b: значение ограничений\n
    Returns:
        * bool: True — если решение допустимо, иначе — False
    """
    return A @ x == b # квадрат евклидовой нормы решения должен быть ≤ 1

In [15]:
x = np.array([[1],[2],[3]])
# x * np.log2(x)

In [16]:
func_hessian(x)

array([[1.44269504],
       [0.72134752],
       [0.48089835]])

# 1) Исследование задачи на выпуклость. Необходимые условия минимума. Двойственная задача.

Данная задача считается решаемой тогда и только тогда, когда $rank(A) = p < n$.

## Выпуклость.

**Определения:**
1) Функция $f(x)$ считается ***convex*** (выпуклой вниз), если для $∀x,y ∈ X ⊂ R^n, ∀γ∈$ отрузку $[0, 1]$ выполняется неравенство: $f(γx + (1-γ)y) ≤ γf(x) + (1-γ)f(y)$. Другими словами — функция выпукла, если любая хорда, соединяющая две точки функции, лежит не ниже самой функции. <br>
![Определение выпуклой вниз функции](./images/convex.png)

In [31]:
for i in range(N): # идём по числу тест-кейсов
    x, y = np.random.rand(DIM, 1), np.random.rand(DIM, 1) # случайно генерируем точки x и y в пространстве размерности DIM ((DIM, 1) — для вектора-столбца)
    γ = np.random.uniform(low=0, high=1) # случайное соотношение x и y (равномерное от 0 до 1)
    if not func(γ*x+(1-γ)*y) <= γ * func(x) + (1-γ) * func(y): # если условие выпуклости нарушено
        raise Exception("Условие выпуклости нарушено!") # выкидываем исключение

print("Функция прошла проверку на выпуклость!")

Функция прошла проверку на выпуклость!


Под ограничением $Ax=b$ понимается пересечение плоскостей (при этом таких плоскостей меньше, чем размерность пространства, $m<n$). Каждая плоскость является выпуклым множеством. Очевидно, что пересечение выпуклых множеств будет выпуклым.

![Определение выпуклой вниз функции](./images/constraints_intersection.png)

## Условие минимума.

Необходимое условие минимума функции: если функция $f(x)$ имеет минимум в точке $х = а$, то в этой точке производная либо **равна нулю**, либо **не существует** (равна бесконечности).

## Двойственная задача.

Построим двойственную задачу по аналогии из предыдущей лабораторной работу.

Изначально целевая функция имеет вид: $f(x) = \sum_{i=1}^{n}x_i log_2 x_i = x^T log_2 x\to min$. <br>
А ограничение: $Ax=b$.

Тогда её функция Лагранжа $L(x, λ)$ имеет вид: 
* $L(x, λ) = x^T log_2 x + λ^T(Ax - b)$.

Производная функции Лагранжа $\frac{dL(x, λ)}{dx}$:
* $\frac{dL(x, λ)}{dx} = log_2 x + \frac{1}{ln2} + A^Tλ = 0$.

Выражаем значение $x^*$:
* $log_2 x^* + \frac{1}{ln2} + A^Tλ = 0$
* $log_2 x^* = -(\frac{1}{ln2} + A^Tλ)$
* $x^* = 2^{-(\frac{1}{ln2} + A^Tλ)}$

Подставляем полученное значение $x^*$ в $L(x, λ)$:
* $g(λ) = (2^{-(\frac{1}{ln2} + A^Tλ)})^T (-(\frac{1}{ln2} + A^Tλ)) + λ(A2^{-(\frac{1}{ln2} + A^Tλ)} - b) = -2^{-(\frac{1}{ln2} + λ^TA)} (\frac{1}{ln2} + A^Tλ) + λ(A2^{-(\frac{1}{ln2} + A^Tλ)} - b)$

Таким образом ***двойственная задача*** выглядит следующим образом:
* Целевая функция: .
* Ограничений нет.

In [36]:
A = np.array([[3, 4, 1], [2, 5, 7]])
λ = np.array([[4], [8]])
2 ** (-(1/np.log(2) + A.T @ λ)).T

array([[1.37045771e-09, 5.10535282e-18, 3.19084551e-19]])

In [37]:
2 ** (-(1/np.log(2) + λ.T @ A))

array([[1.37045771e-09, 5.10535282e-18, 3.19084551e-19]])

# 2) Генерация и решение тестовых примеров с помощью встроенных методов.

# 3) Реализация и тестирование метода Ньютона.